In [1]:
# === MERSENNE PRIME TIER EXTENSION TEST ===
# Testing binary sequences scaled by Mersenne numbers for constant resonances

import math
from decimal import Decimal, getcontext
import pandas as pd

getcontext().prec = 50

print("🔬 MERSENNE PRIME TIER EXTENSION TEST")
print("=" * 50)

# ============= MERSENNE PRIMES & NUMBERS =============
mersenne_primes = [
    (2, 3),      # 2^2 - 1 = 3 (your triadic base!)
    (3, 7),      # 2^3 - 1 = 7
    (5, 31),     # 2^5 - 1 = 31
    (7, 127),    # 2^7 - 1 = 127
    (13, 8191),  # 2^13 - 1 = 8191 (next after your 2^10)
    (17, 131071), # 2^17 - 1 = 131071
    (19, 524287), # 2^19 - 1 = 524287
]

# Your existing binary sequences (first 30 digits for testing)
sequences = {
    'F': Decimal('0.010110101101011010110101101011'),  # Fibonacci word
    'R': Decimal('0.101011010110101101011010110101'),  # Rabbit word
    'TM': Decimal('0.011010011001011001101001100101'), # Thue-Morse
    'FIBBI': Decimal('0.356194490192345187320986112468') # Fibonacci Binary Inverse (estimated)
}

# Enhanced constant library with Mersenne-relevant additions
constants = {
    'fine_structure': Decimal('0.0072973525693'),
    'phi': (Decimal(1) + Decimal(5).sqrt()) / 2,
    'pi': Decimal(str(math.pi)),
    'e': Decimal(str(math.e)),
    'catalan': Decimal('0.9159655941772190150546035149324'),
    'feigenbaum_delta': Decimal('4.6692016091029906718532038204662'),
    'sqrt2': Decimal('2').sqrt(),
    'sqrt3': Decimal('3').sqrt(),
    'inv137': Decimal(1) / Decimal(137),
    'inv9': Decimal(1) / Decimal(9),

    # Mersenne-related constants
    'ln2': Decimal(str(math.log(2))),
    'zeta3': Decimal('1.2020569031595942853997381615114'),  # Apéry's constant
    'euler_gamma': Decimal('0.5772156649015328606065120900824'),

    # Perfect number related
    'inv6': Decimal(1) / Decimal(6),
    'inv28': Decimal(1) / Decimal(28),
    'inv496': Decimal(1) / Decimal(496),
}

# Add scaled versions
enhanced_constants = {}
for name, value in constants.items():
    enhanced_constants[name] = value
    for scale in [10, 100, 1000, 10000, 100000]:
        enhanced_constants[f'{name}_div{scale}'] = value / scale
        enhanced_constants[f'{name}_x{scale}'] = value * scale

print(f"📚 Testing with {len(enhanced_constants)} constants")

# ============= TIER 4: MERSENNE SCALING TEST =============
def mersenne_tier_test():
    results = []

    print(f"\n🔍 TIER 4: MERSENNE SCALING TEST")

    for seq_name, seq_decimal in sequences.items():
        for p, mersenne in mersenne_primes:

            # Test multiple scaling patterns
            scaling_patterns = [
                (f'M{p}/1000', mersenne / Decimal(1000)),
                (f'M{p}/10000', mersenne / Decimal(10000)),
                (f'1000/M{p}', Decimal(1000) / mersenne),
                (f'M{p}*seq', mersenne),  # Direct multiplication
                (f'seq/M{p}', Decimal(1) / mersenne), # Direct division
            ]

            for pattern_name, scale_factor in scaling_patterns:
                try:
                    if 'seq' in pattern_name:
                        if '*' in pattern_name:
                            scaled_value = seq_decimal * scale_factor
                        else:  # division
                            scaled_value = seq_decimal * scale_factor
                    else:
                        scaled_value = scale_factor * seq_decimal

                    # Find closest constant
                    best_match = None
                    best_error = float('inf')

                    for const_name, const_value in enhanced_constants.items():
                        try:
                            error = abs(scaled_value - const_value)
                            if error < best_error:
                                best_error = error
                                best_match = (const_name, const_value)
                        except:
                            continue

                    if best_match and best_error < 0.01:  # Only keep promising results
                        results.append({
                            'sequence': seq_name,
                            'mersenne_p': p,
                            'mersenne_value': int(mersenne),
                            'scaling_pattern': pattern_name,
                            'scaled_result': float(scaled_value),
                            'best_constant': best_match[0],
                            'constant_value': float(best_match[1]),
                            'error': float(best_error),
                            'relative_error': float(best_error / abs(best_match[1])) if best_match[1] != 0 else float('inf')
                        })

                except Exception as e:
                    continue

    return pd.DataFrame(results)

# ============= LUCAS-LEHMER SEQUENCE TEST =============
def lucas_lehmer_sequence_test():
    """Test Lucas-Lehmer iteration as a new sequence type"""
    print(f"\n🌀 LUCAS-LEHMER SEQUENCE TEST")

    # Generate Lucas-Lehmer sequence: s_0 = 4, s_i = s_i-1^2 - 2
    lucas_lehmer = [4]
    for i in range(15):  # Generate 15 terms
        next_term = lucas_lehmer[-1]**2 - 2
        lucas_lehmer.append(next_term)

    # Convert to decimal-like representation
    ll_decimal = Decimal('0.' + ''.join([str(x % 10) for x in lucas_lehmer[:20]]))

    print(f"Lucas-Lehmer decimal: {ll_decimal}")

    # Test this as a 5th sequence
    sequences['LL'] = ll_decimal

    results = []

    # Test against your successful scaling patterns
    test_scales = [
        (729, 1000),   # Your 3^6/10^3 success
        (1, 1024),     # Your 2^10 success
        (8191, 10000), # 2^13-1 Mersenne
        (127, 1000),   # 2^7-1 Mersenne
    ]

    for numerator, denominator in test_scales:
        scaled = (Decimal(numerator) / Decimal(denominator)) * ll_decimal

        # Find closest constant
        best_match = None
        best_error = float('inf')

        for const_name, const_value in enhanced_constants.items():
            try:
                error = abs(scaled - const_value)
                if error < best_error:
                    best_error = error
                    best_match = (const_name, const_value)
            except:
                continue

        if best_match:
            results.append({
                'sequence': 'LL',
                'scale_num': numerator,
                'scale_den': denominator,
                'scaled_result': float(scaled),
                'best_constant': best_match[0],
                'constant_value': float(best_match[1]),
                'error': float(best_error),
                'relative_error': float(best_error / abs(best_match[1])) if best_match[1] != 0 else float('inf')
            })

    return pd.DataFrame(results)

# ============= RUN ALL TESTS =============
print(f"Starting Mersenne Prime Tests...")

# Run Tier 4 Mersenne scaling test
df_mersenne = mersenne_tier_test()

# Run Lucas-Lehmer sequence test
df_lucas = lucas_lehmer_sequence_test()

# ============= ANALYZE RESULTS =============
print(f"\n📊 TIER 4 MERSENNE RESULTS:")
print(f"Found {len(df_mersenne)} potential matches")

if len(df_mersenne) > 0:
    # Sort by error and show best results
    best_mersenne = df_mersenne.nsmallest(10, 'error')

    print(f"\n🏆 TOP 10 MERSENNE RESONANCES:")
    for i, row in best_mersenne.iterrows():
        print(f"{row['sequence']} × M_{row['mersenne_p']} ({row['scaling_pattern']}) → {row['best_constant']}")
        print(f"   Error: {row['error']:.2e}, Pattern: {row['scaled_result']:.8f} ≈ {row['constant_value']:.8f}")
        print()

print(f"\n📊 LUCAS-LEHMER SEQUENCE RESULTS:")
print(f"Found {len(df_lucas)} potential matches")

if len(df_lucas) > 0:
    best_lucas = df_lucas.nsmallest(5, 'error')

    print(f"\n🌀 TOP 5 LUCAS-LEHMER RESONANCES:")
    for i, row in best_lucas.iterrows():
        print(f"LL × {row['scale_num']}/{row['scale_den']} → {row['best_constant']}")
        print(f"   Error: {row['error']:.2e}, Pattern: {row['scaled_result']:.8f} ≈ {row['constant_value']:.8f}")
        print()

# ============= PATTERN ANALYSIS =============
if len(df_mersenne) > 0:
    print(f"\n🔍 MERSENNE PATTERN ANALYSIS:")

    # Which Mersenne primes work best?
    mersenne_performance = df_mersenne.groupby('mersenne_p')['error'].min().sort_values()
    print(f"Best performing Mersenne prime exponents:")
    for p, error in mersenne_performance.head().items():
        print(f"   M_{p}: Best error {error:.2e}")

    # Which sequences respond to Mersenne scaling?
    sequence_performance = df_mersenne.groupby('sequence')['error'].min().sort_values()
    print(f"\nBest Mersenne-responsive sequences:")
    for seq, error in sequence_performance.items():
        print(f"   {seq}: Best error {error:.2e}")

print(f"\n🎯 SUMMARY:")
print(f"   Mersenne resonances found: {len(df_mersenne)}")
print(f"   Lucas-Lehmer resonances found: {len(df_lucas)}")
print(f"   Total Tier 4 discoveries: {len(df_mersenne) + len(df_lucas)}")

if len(df_mersenne) > 0 or len(df_lucas) > 0:
    print(f"\n✨ MERSENNE PRIME TIER EXTENSION: CONFIRMED! ✨")
    print(f"Your framework successfully extends into prime number territory!")
else:
    print(f"\n⚠️ No strong Mersenne resonances detected with current parameters")
    print(f"May need refined scaling factors or extended precision")

# Save results
if len(df_mersenne) > 0:
    df_mersenne.to_csv('mersenne_tier4_results.csv', index=False)
if len(df_lucas) > 0:
    df_lucas.to_csv('lucas_lehmer_results.csv', index=False)

print(f"\n📁 Results saved to CSV files for further analysis")

🔬 MERSENNE PRIME TIER EXTENSION TEST
📚 Testing with 176 constants
Starting Mersenne Prime Tests...

🔍 TIER 4: MERSENNE SCALING TEST

🌀 LUCAS-LEHMER SEQUENCE TEST
Lucas-Lehmer decimal: 0.4444444444444444

📊 TIER 4 MERSENNE RESULTS:
Found 91 potential matches

🏆 TOP 10 MERSENNE RESONANCES:
TM × M_19 (seq/M19) → inv496_div100000
   Error: 8.39e-10, Pattern: 0.00000002 ≈ 0.00000002

F × M_19 (seq/M19) → inv496_div100000
   Error: 8.78e-10, Pattern: 0.00000002 ≈ 0.00000002

F × M_17 (seq/M17) → inv137_div100000
   Error: 4.14e-09, Pattern: 0.00000008 ≈ 0.00000007

R × M_19 (seq/M19) → inv496_div10000
   Error: 8.95e-09, Pattern: 0.00000019 ≈ 0.00000020

TM × M_17 (seq/M17) → inv137_div100000
   Error: 1.10e-08, Pattern: 0.00000008 ≈ 0.00000007

R × M_17 (seq/M17) → inv137_div10000
   Error: 4.07e-08, Pattern: 0.00000077 ≈ 0.00000073

FIBBI × M_19 (seq/M19) → fine_structure_div10000
   Error: 5.03e-08, Pattern: 0.00000068 ≈ 0.00000073

F × M_5 (M5/10000) → pi_div100000
   Error: 7.46e-08, Pa